### 测试

In [4]:
from utils.dataset_utils import get_distribution
from instance_selection.parameter.parameter import *  # 导入参数的设定
from instance_selection_encapsulation.operator.init_toolbox import init_toolbox_emosaic
from instance_selection_encapsulation.operator.metrics import calculate_gmean_mauc, calculate_average_gmean_mauc
from instance_selection_encapsulation.operator.genetic_operator import selTournamentNDCD
from instance_selection_encapsulation.operator.ensemble import vote_result_ensembles, ensemble_individuals
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
import scipy.io as sio  # 从.mat文件中读取数据集
import random
import numpy as np
from deap import tools
import warnings

warnings.filterwarnings("ignore")  # 忽略警告

DATASET = Contraceptive  # 数据集名称（包含对应的参数配置）

datasetname = DATASET.DATASETNAME.split('.')[0]

# 加载、划分数据集
mat_data = sio.loadmat(IMBALANCED_DATASET_PATH + DATASET.DATASETNAME)
x = mat_data['X']
y = mat_data['Y'][:, 0]  # mat_data['Y']得到的形状为[n,1]，通过[:,0]，得到形状[n,]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, stratify=y, random_state=RANDOM_SEED)  # 划分数据集
scaler = StandardScaler()  # 数据的标准化
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

unique_elements_all, classes_all, counts_all = get_distribution(y)  # 获取原始数据集分布
unique_elements_train, classes_train, counts_train = get_distribution(y_train)  # 获取训练集分布
unique_elements_test, classes_test, counts_test = get_distribution(y_test)  # 获取测试集分布
print(datasetname + f' instances: {x.shape[0]}')
print(datasetname + f' distribution: {counts_all}')
print(f'trainset distribution: {counts_train}')
print(f'testset distribution: {counts_test}')
model = MLPClassifier(hidden_layer_sizes=(DATASET.HIDDEN_SIZE,), max_iter=DATASET.MAX_ITER,
                      random_state=RANDOM_SEED, learning_rate_init=DATASET.LEARNING_RATE)

model.fit(x_train, y_train)

# 测试模型
y_test_probs = model.predict_proba(x_test)
y_test_preds = model.predict(x_test)

print(calculate_gmean_mauc(y_test_probs, y_test))

Contraceptive instances: 1473
Contraceptive distribution: [629 333 511]
trainset distribution: [440 233 358]
testset distribution: [189 100 153]
(0.481712, 0.719694, array([0.58730159, 0.28      , 0.67973856]))


## E-MOSAIC

In [5]:
toolbox = init_toolbox_emosaic(model, x_train, y_train, n_splits=N_SPLITS-2, random_seed=RANDOM_SEED)  # 初始化toolbox


def main(x_train, y_train, model, balanced_method='balanced'):
    not_replaced = 0
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    logbook = tools.Logbook()
    logbook.header = "gen", "fronts", "ensembles_size", "avg_gmean", "avg_mauc"
    ####################################种群的初始化###########################
    pop = toolbox.population(n=POPSIZE)  # 个体编码默认全为0
    pop = toolbox.init_population(pop, balanced_method=balanced_method)  # 初始化种群中的个体
    toolbox.evaluate(pop)  # 计算个体的适应度
    save_ensembles = ensemble_individuals(pop, model, x_train, y_train)  # 保存最优的集成分类器
    save_gmean, save_mauc = calculate_average_gmean_mauc(pop)
    ####################################种群的迭代#################################################
    for gen in range(1, NGEN + 1):
        offspring = selTournamentNDCD(pop, POPSIZE, tournsize=3)  # 锦标赛选择（1、先根据非支配排序的等级2、再根据拥挤距离）
        offspring = [toolbox.clone(ind) for ind in offspring]
        for i in range(0, len(offspring) - 1, 2):
            if random.random() <= CXPB:
                offspring[i], offspring[i + 1] = toolbox.mate(offspring[i], offspring[i + 1])  # 单点交叉
            offspring[i] = toolbox.mutate(offspring[i], MR)[0]  # 二进制反转突变
            offspring[i + 1] = toolbox.mutate(offspring[i + 1], MR)[0]  # 二进制反转突变
            del offspring[i].fitness.values, offspring[i + 1].fitness.values
        #############################################################合并、去重#####################################################
        offspring = toolbox.individuals_constraints(offspring)  # 限制每个类至少有一个实例被选择
        pop = pop + offspring  # 种群的合并
        pop, _ = toolbox.remove_duplicates(pop)  # 去重
        while len(pop) < POPSIZE:  # 保证种群大小为POPSIZE
            add_individual = []
            num_add = POPSIZE - len(pop)
            for i in range(0, num_add):
                index = random.randint(0, len(offspring) - 1)  # 在0-len(offspring)范围内随机产生一个索引
                offspring[index] = toolbox.mutate(offspring[index], MR)[0]  # 选择index对应的个体进行突变
                del offspring[index].fitness.values
                add_individual.append(offspring[index])
            add_individual = toolbox.individuals_constraints(add_individual)  # 限制每个类至少有一个实例被选择
            pop = pop + add_individual  # 种群的合并
            pop, _ = toolbox.remove_duplicates(pop)  # 去重
        pop = toolbox.individuals_constraints(pop) # 限制每个类至少有5个实例被选择
        toolbox.evaluate(pop)  # 计算新种群适应度
        ###############################################得到pareto_fronts############################################
        pop, pareto_fronts = toolbox.select(pop, POPSIZE)
        ensembles = pop  # 集成的个体
        curr_gmean, curr_mauc = calculate_average_gmean_mauc(ensembles)  # 计算的是所有个体的metrics平均值
        if curr_gmean >= save_gmean and curr_mauc >= save_mauc:
            save_ensembles = ensembles
            not_replaced = 0
        else:
            not_replaced += 1
        if not_replaced >= STOP_SIGN:
            break  # 迭代结束
        record = stats.compile(pop)
        logbook.record(gen=gen, fronts=len(pareto_fronts), ensembles_size=len(ensembles),
                       avg_gmean=curr_gmean, avg_mauc=curr_mauc, **record)
        # print(logbook.stream)
    ensemble_classifiers = ensemble_individuals(save_ensembles, model, x_train, y_train)
    return ensemble_classifiers


if __name__ == "__main__":
    print("*****************算法开始执行：******************")
    num_run = 40  # 运行次数
    ensembles_results = [[] for _ in range(num_run)]
    for i in range(num_run):
        ensemble_classifiers = main(x_train, y_train, model=model)
        vote_pred_prob = vote_result_ensembles(ensemble_classifiers, x_test)  # 默认预测结果是软标签
        gmean, mauc, recall_per_class = calculate_gmean_mauc(vote_pred_prob, y_test)
        ensembles_results[i] = [gmean, mauc]
        print(f"第{i + 1}次执行：Gmean：{gmean}，mAUC：{mauc}")
    print("*****************算法执行结束！******************")
    ensembles_result_mean = np.mean(ensembles_results, axis=0)
    print(f'集成分类结果（平均值）：{ensembles_result_mean}')

*****************算法开始执行：******************
第1次执行：Gmean：0.536333，mAUC：0.740346
第2次执行：Gmean：0.548047，mAUC：0.742039
第3次执行：Gmean：0.525883，mAUC：0.737494
第4次执行：Gmean：0.529833，mAUC：0.739639
第5次执行：Gmean：0.532404，mAUC：0.730253
第6次执行：Gmean：0.542044，mAUC：0.733996
第7次执行：Gmean：0.53592，mAUC：0.736641
第8次执行：Gmean：0.536423，mAUC：0.732749
第9次执行：Gmean：0.525652，mAUC：0.7377
第10次执行：Gmean：0.522436，mAUC：0.734084
第11次执行：Gmean：0.540024，mAUC：0.731242
第12次执行：Gmean：0.557137，mAUC：0.744898
第13次执行：Gmean：0.546968，mAUC：0.736355
第14次执行：Gmean：0.516544，mAUC：0.734833
第15次执行：Gmean：0.523139，mAUC：0.73387
第16次执行：Gmean：0.558589，mAUC：0.740808
第17次执行：Gmean：0.545678，mAUC：0.743164
第18次执行：Gmean：0.517383，mAUC：0.731874
第19次执行：Gmean：0.556066，mAUC：0.737227
第20次执行：Gmean：0.546516，mAUC：0.73639
第21次执行：Gmean：0.536383，mAUC：0.744795
第22次执行：Gmean：0.544257，mAUC：0.738489
第23次执行：Gmean：0.562393，mAUC：0.737694
第24次执行：Gmean：0.55431，mAUC：0.740992
第25次执行：Gmean：0.54603，mAUC：0.744285
第26次执行：Gmean：0.555475，mAUC：0.738128
第27次执行：Gmean：0.505679，mAUC：0.733278
第

## 写入到Excel 

In [6]:
from utils.excel_utils import save_to_excel_2

columns = ['Gmean', 'MAUC']
save_path = 'C:/Users/Lenovo/Desktop/Stratified/7-3/E-MOSAIC/' + datasetname + '/'
filename = datasetname
save_to_excel_2(save_path, filename, columns, ensembles_results)

Excel 文件已保存至: C:/Users/Lenovo/Desktop/Stratified/7-3/E-MOSAIC/Contraceptive/Contraceptive.xlsx


'C:/Users/Lenovo/Desktop/Stratified/7-3/E-MOSAIC/Contraceptive/Contraceptive.xlsx'